<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/sft_peft_lora_gemma_2b_with_additional_tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 训练 PEFT 模型时添加新标记到嵌入层和分词器


在这个示例中，我们将学习如何在向 tokenizer 和模型添加新标记时训练 LoRA 模型。
这是一个常见的用例，用于以下情况：
1. 使用新增标记进行指令微调，例如  `<|user|>`, `<|assistant|>`, `<|system|>`, `</s>`, `<s>`，以正确格式化对话。
2. 在特定语言上微调，在该语言的数据集上微调LLM时，添加语言特定的标记，例如添加韩语标记到词汇表中。
3. 在指令微调中返回特定格式的输出，以启用代理行为，例如新增标记 `<|FUNCTIONS|>`, `<|BROWSE|>`, `<|TEXT2IMAGE|>`, `<|ASR|>`, `<|TTS|>`, `<|GENERATECODE|>`, `<|RAG|>`。

在这种情况下，您需要将嵌入模块添加到 LORA 的 `target_modules` 中。PEFT 将负责保存嵌入层，其中包含新添加标记的权重，以及在特定初始化的嵌入层权重上训练的适配器权重。

In [16]:
!pip install -q peft transformers datasets accelerate wandb dataclass_csv

In [17]:
import os

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["WANDB_PROJECT"] = "PeftExamples"
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import transformers
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    TrainingArguments,
    Trainer,
    default_data_collator,
)
import torch
from dataclasses import dataclass, field
from typing import Optional
from dataclass_csv import DataclassReader
from torch.utils.data import Dataset, DataLoader

from enum import Enum

## Prepare Model and Tokenizer

现在，我们将添加 27 个新标记，并替换模型中现有的 pad、bos 和 eos 标记。






In [18]:
class SpecialTokens(str, Enum):
    begin_target = "<|begintarget|>"
    end_target = "<|endtarget|>"
    begin_context = "<|begincontext|>"
    end_context = "<|endcontext|>"
    system = "<|system|>"
    user = "<|user|>"
    begin_last_user_utterance = "<|beginlastuserutterance|>"
    end_last_user_utterance = "<|endlastuserutterance|>"
    begin_dsts = "<|begindsts|>"
    end_dsts = "<|enddsts|>"
    begin_dst = "<|begindst|>"
    end_dst = "<|enddst|>"
    begin_belief = "<|beginbelief|>"
    end_belief = "<|endbelief|>"
    begin_response = "<|beginresponse|>"
    end_response = "<|endresponse|>"
    begin_action = "<|beginaction|>"
    end_action = "<|endaction|>"
    begin_user_action = "<|beginuseraction|>"
    end_user_action = "<|enduseraction|>"
    sys_actions = "<|sysactions|>"
    begin_intent = "<|beginintent|>"
    end_intent = "<|endintent|>"
    begin_requested_slots = "<|beginrequestedslots|>"
    end_requested_slots = "<|endrequestedslots|>"
    pad_token = "<|pad|>"
    bos_token = "<|startoftext|>"

    @classmethod
    def list(cls):
        return [c.value for c in cls]

我们将微调 Mistral-7B 模型。让我们加载分词器并添加特殊标记，然后加载基础模型并调整嵌入层大小以容纳新增标记。

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [20]:
model_name = "google/gemma-2b"
#model_name = "google/gemman-2b-it"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    pad_token=SpecialTokens.pad_token.value,
    bos_token=SpecialTokens.bos_token.value,
    eos_token=SpecialTokens.end_target.value,
    additional_special_tokens=SpecialTokens.list(),
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True
    # use_flash_attention_2=True, # leading to an error
).to(device)
print(model)
model.resize_token_embeddings(len(tokenizer))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaRM

Embedding(256027, 2048)

In [21]:
print(model)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256027, 2048)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaRMSNorm()
  )
  (

In [22]:
print(tokenizer)

GemmaTokenizerFast(name_or_path='google/gemma-2b', vocab_size=256000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<|startoftext|>', 'eos_token': '<|endtarget|>', 'unk_token': '<unk>', 'pad_token': '<|pad|>', 'additional_special_tokens': ['<|begintarget|>', '<|endtarget|>', '<|begincontext|>', '<|endcontext|>', '<|system|>', '<|user|>', '<|beginlastuserutterance|>', '<|endlastuserutterance|>', '<|begindsts|>', '<|enddsts|>', '<|begindst|>', '<|enddst|>', '<|beginbelief|>', '<|endbelief|>', '<|beginresponse|>', '<|endresponse|>', '<|beginaction|>', '<|endaction|>', '<|beginuseraction|>', '<|enduseraction|>', '<|sysactions|>', '<|beginintent|>', '<|endintent|>', '<|beginrequestedslots|>', '<|endrequestedslots|>', '<|pad|>', '<|startoftext|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, sp

## Apply LoRA

**注**：IA3和LoRA差不多，增加了前馈层的参数权重微调，LoRA关注在embedding,self attention， lm_head层

In [23]:
config = LoraConfig(
    r=64, lora_alpha=128, lora_dropout=0.0, target_modules=["embed_tokens", "lm_head", "q_proj", "v_proj"]
)
#config = IA3Config(task_type=TaskType.SEQ_CLS, target_modules=["embed_tokens", "lm_head", "q_proj", "v_proj"], feedforward_modules=["down_proj"])

print(config)
model = get_peft_model(model, config)
print(model.print_trainable_parameters())
print(model)

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=None, inference_mode=False, r=64, target_modules={'lm_head', 'q_proj', 'v_proj', 'embed_tokens'}, lora_alpha=128, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)
trainable params: 40,406,400 || all params: 2,546,634,112 || trainable%: 1.5866590261082625
None
PeftModel(
  (base_model): LoraModel(
    (model): GemmaForCausalLM(
      (model): GemmaModel(
        (embed_tokens): lora.Embedding(
          (base_layer): Embedding(256027, 2048)
          (lora_dropout): ModuleDict(
            (default): Identity()
          )
          (lora_A): ModuleDict()
          (lora_B): ModuleDict()
          (lora_embedding_

`LoraConfig` 的配置，用于配置 LoRA（Low-Rank Adaptation）的相关选项。LoRA 是一种低秩适应技术，用于减少大型语言模型的参数量，并提高模型的效率和性能。以下是各个参数的含义解释：

1. `peft_type`: PEFT（Parameter-Efficient Fine-Tuning）类型。在这里，设置为 `PeftType.LORA`，表示使用 LoRA 进行微调。

2. `auto_mapping`: 自动映射。用于指定是否自动映射预训练模型的参数以适应新任务。

3. `base_model_name_or_path`: 基础模型名称或路径。用于指定要微调的基础语言模型的名称或路径。

4. `revision`: 模型修订版本。

5. `task_type`: 任务类型。在这里，未指定任务类型。

6. `inference_mode`: 推理模式。用于指定是否在推理时使用 LoRA。

7. `r`: LoRA 中的秩（rank）参数。用于控制低秩适应的参数量。

8. `target_modules`: 目标模块。指定了要应用低秩适应的模块名称集合。

9. `lora_alpha`: LoRA 中的 alpha 参数。用于控制低秩适应的强度。

10. `lora_dropout`: LoRA 中的 dropout 参数。用于控制低秩适应的正则化。

11. `fan_in_fan_out`: 是否使用扇入/扇出初始化。

12. `bias`: 偏置类型。指定了在低秩适应中使用的偏置类型。

13. `use_rslora`: 是否使用 RSLora。

14. `modules_to_save`: 要保存的模块列表。

15. `init_lora_weights`: 是否初始化 LoRA 权重。

16. `layers_to_transform`: 要转换的层列表。

17. `layers_pattern`: 层模式。

18. `rank_pattern`: 秩模式。

19. `alpha_pattern`: Alpha 模式。

20. `megatron_config`: Megatron 配置。

21. `megatron_core`: Megatron 核心。

22. `loftq_config`: LoFTQ 配置。

23. `use_dora`: 是否使用 DoRA。

24. `layer_replication`: 层复制。

这些参数用于配置 LoRA 过程中的各个方面，包括秩参数、模块选择、初始化方法等。通过调整这些参数，可以控制 LoRA 的行为，以满足特定任务和硬件环境的需求。

## Preapre Dataset

In [24]:
from datasets import load_dataset

dataset = load_dataset("smangrul/assistant_chatbot_dataset")
dataset = dataset["train"].train_test_split(0.2)

text_column = "context"
label_column = "target"
max_length = 512


def preprocess_function(examples):
    batch_size = len(examples[text_column])
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(examples[text_column])
    labels = tokenizer(text_target=targets, add_special_tokens=False)  # don't add bos token because we concatenate with inputs
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.eos_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = model_inputs["input_ids"][i][:max_length]
        model_inputs["attention_mask"][i] = model_inputs["attention_mask"][i][:max_length]
        labels["input_ids"][i] = labels["input_ids"][i][:max_length]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

train_dataset = processed_datasets["train"]

Running tokenizer on dataset:   0%|          | 0/986 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/247 [00:00<?, ? examples/s]

In [25]:
print(dataset,train_dataset)
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['dialog_id', 'turn_id', 'context', 'target'],
        num_rows: 986
    })
    test: Dataset({
        features: ['dialog_id', 'turn_id', 'context', 'target'],
        num_rows: 247
    })
}) Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 986
})
{'dialog_id': '1_00081', 'turn_id': 9, 'context': '<|begincontext|><|user|>I have a date later and I am looking for a good place to eat.<|system|>What type of cuisine are you in the mood for? Which city will you be going to eat?<|user|>I will be in Berkeley and am in the mood for some Breakfast type of food.<|system|>I found 1 match called the Venus Restaurant in Berkeley.<|user|>Can you give me their address?<|system|>You can find them at 2327 Shattuck Avenue.<|user|>That works out great.<|system|>Would you like to go ahead and make a reservation?<|user|>Yes please. Go ahead and place the reservation.<|system|>What time did you have in mind?<|user|>It is fo

In [26]:
train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=8, pin_memory=True
)

In [27]:
next(iter(train_dataloader))

{'input_ids': tensor([[256002, 256002, 256002,  ..., 256017, 256001, 256001],
         [256002, 256002, 256002,  ..., 256017, 256001, 256001],
         [256002, 256002, 256002,  ..., 256017, 256001, 256001],
         ...,
         [256002, 256002, 256002,  ..., 256017, 256001, 256001],
         [256002, 256002, 256002,  ..., 256017, 256001, 256001],
         [256002, 256002, 256002,  ..., 256017, 256001, 256001]]),
 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         ...,
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1]]),
 'labels': tensor([[  -100,   -100,   -100,  ..., 256017, 256001, 256001],
         [  -100,   -100,   -100,  ..., 256017, 256001, 256001],
         [  -100,   -100,   -100,  ..., 256017, 256001, 256001],
         ...,
         [  -100,   -100,   -100,  ..., 256017, 256001, 256001],
         [  -100,   -100,   -100,  ..., 256017, 25600

In [28]:
tokenizer.decode(train_dataset[0]["input_ids"])

'<|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad

# Train the model

In [29]:
training_args = TrainingArguments(
    output_dir="sft_peft_lora_gemma_2b_with_added_tokens",
    num_train_epochs=2,
    save_total_limit=5,
    per_device_train_batch_size=8,
    warmup_steps=10,
    weight_decay=0.0001,
    dataloader_drop_last=True,
    #bf16=True,
    bf16=False,
    logging_steps=10,
    learning_rate=1e-5,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    remove_unused_columns=False,
    hub_model_id="smangrul/mistral_lora_clm_with_added_tokens",
    push_to_hub=True,
    hub_private_repo=True,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=default_data_collator,
)
# model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss


KeyboardInterrupt: 

# 在评估数据集的样本上检查模型输出

In [ ]:
import random

i = random.randint(0, len(dataset["test"]))
context = dataset["test"][i]["context"]

batch = tokenizer(context, return_tensors="pt")
batch = {k: v.to(device) for k, v in batch.items()}
model.eval()
output_tokens = model.generate(
    **batch,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.2,
    top_p=0.95,
    top_k=50,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)
target_predicted = tokenizer.decode(output_tokens[0], skip_special_tokens=False).split("<|endcontext|>")[1]
target = dataset["test"][i]["target"]
print(f"{context=} \n\n {target_predicted=} \n\n {target=}")

context="<|begincontext|><|user|>Can you find me a place to eat please?<|system|>Where at? And what kind of cuisine are you craving?<|user|>Somewhere in SF, and I am really craving Thai food at the moment!<|system|>I found a bunch of restaurants, there's actually 10 that you might like in San Francisco, one of them being Baan Thai House & Wine Bar<|user|>How can I reach them? And what's their address?<|system|>You can reach them by phone at 415-379-4505 and visit them at 534 Irving Street<|beginlastuserutterance|>Great, that restaurant sounds good<|endlastuserutterance|><|endcontext|>" 

 target_predicted='<|begintarget|><|begindsts|><|begindst|><|beginintent|> FindRestaurants<|endintent|><|beginbelief|> Restaurants^city->SF~San Francisco|Restaurants^cuisine->Thai|Restaurants^restaurant_name->Baan Thai House & Wine Bar<|endbelief|><|enddst|><|enddsts|><|beginuseraction|> REQUEST->Restaurants^phone_number~|REQUEST->Restaurants^street_address~<|enduseraction|><|beginaction|> INFORM->Rest

# 保存适配器模型

当lora层应用于嵌入层时，相应的基础模型嵌入层也会被保存。

In [ ]:
trainer.push_to_hub()
trainer.model.push_to_hub(training_args.output_dir)

# 检查模型是否按预期加载并生成合理的输出

In [ ]:
from peft import PeftModel

inference_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    # use_flash_attention_2=True,
)
inference_model.resize_token_embeddings(len(tokenizer))

inference_model = PeftModel.from_pretrained(inference_model, "weege007/sft_peft_lora_gemma_2b_with_added_tokens")
inference_model.to(device)
inference_model.eval()

output_tokens = inference_model.generate(
    **batch,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.2,
    top_p=0.95,
    top_k=50,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

target_predicted = tokenizer.decode(output_tokens[0], skip_special_tokens=False).split("<|endcontext|>")[1]
print(f"{context=} \n\n {target_predicted=} \n\n {target=}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

context="<|begincontext|><|user|>Can you find me a place to eat please?<|system|>Where at? And what kind of cuisine are you craving?<|user|>Somewhere in SF, and I am really craving Thai food at the moment!<|system|>I found a bunch of restaurants, there's actually 10 that you might like in San Francisco, one of them being Baan Thai House & Wine Bar<|user|>How can I reach them? And what's their address?<|system|>You can reach them by phone at 415-379-4505 and visit them at 534 Irving Street<|beginlastuserutterance|>Great, that restaurant sounds good<|endlastuserutterance|><|endcontext|>" 

 target_predicted='<|begintarget|><|begindsts|><|begindst|><|beginintent|> FindRestaurant<|endintent|><|beginbelief|> Restaurants^city->SF~San Francisco|Restaurants^cuisine->Thai|Restaurants^restaurant_name->Baan Thai House & Wine Bar<|endbelief|><|enddst|><|enddsts|><|beginuseraction|> REQUEST->Restaurants^phone_number~|REQUEST->Restaurants^street_address~<|enduseraction|><|beginaction|> INFORM->Resta

## 使用llama.cpp 测试下 gguf 支持

需要把新的 tokenizer.json tokenizer.model 格式化到 gguf中

## 使用 gemma.cpp 测试下

gemma.cpp 模型权重和分词器加载